In [1]:
from torch import nn
from torch.nn import functional as F
from tqdm import tqdm

import torch
import math

In [2]:
pad_id = 0
vocab_size = 100

data = [
  [62, 13, 47, 39, 78, 33, 56, 13, 39, 29, 44, 86, 71, 36, 18, 75],
  [60, 96, 51, 32, 90],
  [35, 45, 48, 65, 91, 99, 92, 10, 3, 21, 54],
  [75, 51],
  [66, 88, 98, 47],
  [21, 39, 10, 64, 21],
  [98],
  [77, 65, 51, 77, 19, 15, 35, 19, 23, 97, 50, 46, 53, 42, 45, 91, 66, 3, 43, 10],
  [70, 64, 98, 25, 99, 53, 4, 13, 69, 62, 66, 76, 15, 75, 45, 34],
  [20, 64, 81, 35, 76, 85, 1, 62, 8, 45, 99, 77, 19, 43]
]

In [3]:
def padding(data):
    max_len = len(max(data, key=len))
    print(f"Maximum sequence length: {max_len}")
    
    for i, seq in enumerate(tqdm(data)):
        if len(seq) < max_len:
            data[i] = seq + [pad_id] * (max_len - len(seq))
    return data, max_len

In [4]:
data, max_len = padding(data)

100%|██████████| 10/10 [00:00<00:00, 93832.30it/s]

Maximum sequence length: 20


In [5]:
data

[[62, 13, 47, 39, 78, 33, 56, 13, 39, 29, 44, 86, 71, 36, 18, 75, 0, 0, 0, 0],
 [60, 96, 51, 32, 90, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [35, 45, 48, 65, 91, 99, 92, 10, 3, 21, 54, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [75, 51, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [66, 88, 98, 47, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [21, 39, 10, 64, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [98, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [77,
  65,
  51,
  77,
  19,
  15,
  35,
  19,
  23,
  97,
  50,
  46,
  53,
  42,
  45,
  91,
  66,
  3,
  43,
  10],
 [70, 64, 98, 25, 99, 53, 4, 13, 69, 62, 66, 76, 15, 75, 45, 34, 0, 0, 0, 0],
 [20, 64, 81, 35, 76, 85, 1, 62, 8, 45, 99, 77, 19, 43, 0, 0, 0, 0, 0, 0]]

In [6]:
d_model = 512 #model의 hidden size
num_heads = 8 #head의 개수

In [11]:
embedding = nn.Embedding(vocab_size, d_model)

batch = torch.LongTensor(data)
batch_emb = embedding(batch)

In [13]:
print(batch_emb)
print(batch_emb.shape)

tensor([[[-0.0690,  0.9569,  0.0342,  ...,  0.0632,  0.2650,  0.1710],
         [-0.1351, -1.7390, -0.6677,  ...,  1.1584,  0.1202, -0.6042],
         [-0.2664, -0.5098, -1.0920,  ...,  1.3691,  0.1031,  0.6187],
         ...,
         [ 0.1616,  1.9206, -1.2345,  ..., -0.2741, -1.0108,  0.3288],
         [ 0.1616,  1.9206, -1.2345,  ..., -0.2741, -1.0108,  0.3288],
         [ 0.1616,  1.9206, -1.2345,  ..., -0.2741, -1.0108,  0.3288]],

        [[-0.3157,  0.6164,  0.2753,  ..., -0.0811,  1.3366, -0.6698],
         [ 0.7110, -1.2910,  0.8190,  ...,  0.9541, -1.5577, -0.5547],
         [ 0.2349,  1.7097, -1.2290,  ..., -1.7882,  2.2449, -0.6559],
         ...,
         [ 0.1616,  1.9206, -1.2345,  ..., -0.2741, -1.0108,  0.3288],
         [ 0.1616,  1.9206, -1.2345,  ..., -0.2741, -1.0108,  0.3288],
         [ 0.1616,  1.9206, -1.2345,  ..., -0.2741, -1.0108,  0.3288]],

        [[-0.5883,  1.3864, -0.0755,  ..., -0.3122,  0.9327, -0.4524],
         [ 1.5826,  1.3885, -1.3150,  ..., -0

In [14]:
w_q = nn.Linear(d_model, d_model)
w_k = nn.Linear(d_model, d_model)
w_v = nn.Linear(d_model, d_model)

In [15]:
w_0 = nn.Linear(d_model, d_model)

In [16]:
q = w_q(batch_emb)  # (B, L, d_model)
k = w_k(batch_emb)  # (B, L, d_model)
v = w_v(batch_emb)  # (B, L, d_model)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 20, 512])
torch.Size([10, 20, 512])
torch.Size([10, 20, 512])


In [21]:
batch_size = q.shape[0]
d_k = d_model // num_heads

q = q.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
k = k.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
v = v.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 20, 8, 64])
torch.Size([10, 20, 8, 64])
torch.Size([10, 20, 8, 64])
torch.Size([10, 20, 64, 8])
torch.Size([10, 20, 64, 8])


In [23]:
q = q.transpose(1, 2)  # (B, num_heads, L, d_k)
k = k.transpose(1, 2)  # (B, num_heads, L, d_k)
v = v.transpose(1, 2)  # (B, num_heads, L, d_k)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 8, 20, 64])
torch.Size([10, 8, 20, 64])
torch.Size([10, 8, 20, 64])


In [26]:
attn_scores = torch.matmul(q, k.transpose(-2,-1)) / math.sqrt(d_k) # (B, num_heads, L, L)
attn_dists = F.softmax(attn_scores, dim=-1)  # (B, num_heads, L, L)

#print(attn_dists)
print(attn_dists.shape)

torch.Size([10, 8, 20, 20])


In [27]:
attn_values = torch.matmul(attn_dists, v)  # (B, num_heads, L, d_k)

print(attn_values.shape)

torch.Size([10, 8, 20, 64])


In [28]:
attn_values = attn_values.transpose(1, 2)  # (B, L, num_heads, d_k)
attn_values = attn_values.contiguous().view(batch_size, -1, d_model)  # (B, L, d_model)

print(attn_values.shape)

torch.Size([10, 20, 512])


In [29]:
outputs = w_0(attn_values)

print(outputs)
print(outputs.shape)

tensor([[[ 0.2144, -0.0449,  0.0276,  ..., -0.0633, -0.0050, -0.1183],
         [ 0.1394, -0.0516,  0.0591,  ..., -0.0700, -0.0338, -0.1512],
         [ 0.1084, -0.0692, -0.0139,  ..., -0.0372,  0.0058, -0.1162],
         ...,
         [ 0.1207, -0.0390, -0.0695,  ..., -0.0745,  0.0317, -0.1047],
         [ 0.1207, -0.0390, -0.0695,  ..., -0.0745,  0.0317, -0.1047],
         [ 0.1207, -0.0390, -0.0695,  ..., -0.0745,  0.0317, -0.1047]],

        [[ 0.1102, -0.1848,  0.1488,  ..., -0.4503, -0.1697, -0.1548],
         [ 0.0801, -0.1791,  0.1592,  ..., -0.4735, -0.1449, -0.2138],
         [ 0.1635, -0.1896,  0.2121,  ..., -0.5277, -0.1292, -0.2206],
         ...,
         [ 0.0897, -0.2068,  0.1630,  ..., -0.4870, -0.1488, -0.1985],
         [ 0.0897, -0.2068,  0.1630,  ..., -0.4870, -0.1488, -0.1985],
         [ 0.0897, -0.2068,  0.1630,  ..., -0.4870, -0.1488, -0.1985]],

        [[ 0.0593, -0.1609,  0.0727,  ..., -0.2531, -0.0963, -0.0767],
         [ 0.1143, -0.1197,  0.1059,  ..., -0

In [30]:
class MultiheadAttention(nn.Module):
  def __init__(self):
    super(MultiheadAttention, self).__init__()

    # Q, K, V learnable matrices
    self.w_q = nn.Linear(d_model, d_model)
    self.w_k = nn.Linear(d_model, d_model)
    self.w_v = nn.Linear(d_model, d_model)

    # Linear transformation for concatenated outputs
    self.w_0 = nn.Linear(d_model, d_model)

  def forward(self, q, k, v):
    batch_size = q.shape[0]

    q = self.w_q(q)  # (B, L, d_model)
    k = self.w_k(k)  # (B, L, d_model)
    v = self.w_v(v)  # (B, L, d_model)

    q = q.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
    k = k.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
    v = v.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)

    q = q.transpose(1, 2)  # (B, num_heads, L, d_k)
    k = k.transpose(1, 2)  # (B, num_heads, L, d_k)
    v = v.transpose(1, 2)  # (B, num_heads, L, d_k)

    attn_values = self.self_attention(q, k, v)  # (B, num_heads, L, d_k)
    attn_values = attn_values.transpose(1, 2).contiguous().view(batch_size, -1, d_model)  # (B, L, num_heads, d_k) => (B, L, d_model)

    return self.w_0(attn_values)

  def self_attention(self, q, k, v):
    attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)  # (B, num_heads, L, L)
    attn_dists = F.softmax(attn_scores, dim=-1)  # (B, num_heads, L, L)

    attn_values = torch.matmul(attn_dists, v)  # (B, num_heads, L, d_k)

    return attn_values

In [31]:
multihead_attn = MultiheadAttention()

outputs = multihead_attn(batch_emb, batch_emb, batch_emb)  # (B, L, d_model)

In [32]:
print(outputs)
print(outputs.shape)

tensor([[[-1.5793e-01, -4.6884e-02, -2.3922e-01,  ..., -6.3330e-02,
          -4.6379e-02,  3.5649e-02],
         [-1.2245e-01, -9.0145e-02, -2.3995e-01,  ..., -8.6596e-03,
          -2.8390e-02,  6.3573e-02],
         [-1.7233e-01, -9.0526e-02, -2.2073e-01,  ...,  4.3520e-03,
          -1.9803e-02,  3.6607e-02],
         ...,
         [-1.6229e-01, -2.6198e-02, -2.0113e-01,  ..., -5.4172e-02,
          -8.7981e-03,  5.7405e-03],
         [-1.6229e-01, -2.6198e-02, -2.0113e-01,  ..., -5.4172e-02,
          -8.7980e-03,  5.7405e-03],
         [-1.6229e-01, -2.6198e-02, -2.0113e-01,  ..., -5.4172e-02,
          -8.7980e-03,  5.7405e-03]],

        [[-3.8079e-01, -2.6607e-01, -4.7060e-01,  ...,  1.5971e-01,
          -1.9926e-01, -4.3120e-02],
         [-4.0115e-01, -2.7004e-01, -4.4659e-01,  ...,  1.1378e-01,
          -2.1741e-01, -3.3329e-02],
         [-3.8535e-01, -2.5117e-01, -4.1696e-01,  ...,  6.8521e-02,
          -1.7563e-01, -4.2442e-02],
         ...,
         [-4.1860e-01, -2